In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

%matplotlib inline

In [2]:
ads_df = pd.read_csv('data/political_tv_ad_archive_entire_dataset.csv')

In [3]:
ads_metadata = pd.read_csv("data/unique_ad_metadata.csv")

In [4]:
ads_df.head(2)

,id,wp_identifier,network,location,program,program_type,start_time,end_time,archive_id,embed_url,sponsors,sponsor_types,race,cycle,subjects,candidates,type,message,date_created
0,1,5643,KLAS,"Las Vegas, NV",8 News Now at 5 PM,news,2016-09-09 00:12:59 UTC,2016-09-09 00:13:29 UTC,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,2016.0,"Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32
1,2,5643,WNCN,"Raleigh-Durham-Fayetteville, NC",North Carolina News at 500PM,news,2016-09-06 21:58:25 UTC,2016-09-06 21:58:55 UTC,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,2016.0,"Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32


In [5]:
ads_metadata.head()

,wp_identifier,archive_id,embed_url,sponsors,sponsor_types,subjects,candidates,type,race,cycle,message,air_count,reference_count,market_count,transcript,date_ingested
0,8403,PolAd_CatherineCortezMasto_jbah8,https://archive.org/embed/PolAd_CatherineCorte...,Catherine Cortez Masto for Senate,Candidate Committee,"Candidate Biography, Bipartisanship, Criminal ...",Catherine Cortez Masto,campaign,NVS2,2016.0,pro,185,0,1,NaN,2016/11/17 5:03:54 UTC
1,8404,PolAd_DonaldTrump_kc0en,https://archive.org/embed/PolAd_DonaldTrump_kc0en,Hillary for America,Multiple,"Nuclear, Candidate Biography, Military, Foreig...",Donald Trump,campaign,PRES,2016.0,con,139,0,1,NaN,2016/11/17 5:02:52 UTC
2,8358,PolAd_DonaldTrump_ncck5,https://archive.org/embed/PolAd_DonaldTrump_ncck5,Hillary for America,Multiple,"Bankruptcy, Candidate Biography, Workers",Donald Trump,campaign,PRES,2016.0,con,195,0,2,NaN,2016/11/14 10:44:42 UTC
3,8359,PolAd_Guns_gv0de,https://archive.org/embed/PolAd_Guns_gv0de,National Rifle Assn,PAC,Guns,NaN,campaign,NaN,NaN,unknown,87,0,3,NaN,2016/11/14 10:44:15 UTC
4,8361,PolAd_Guns_Veterans_Military_z2bvd,https://archive.org/embed/PolAd_Guns_Veterans_...,National Rifle Assn,PAC,"Guns, Veterans, Military",NaN,campaign,NaN,NaN,unknown,54,0,3,NaN,2016/11/14 10:43:51 UTC


# Initial Cleaning

Add filtering here first before adding/changing column types (reduce data size)

In [6]:
# filter on dates, campaign types, potentially remove nulls
ads_df.isnull().sum()

id                   0
wp_identifier        0
network              0
location            25
program             15
program_type         0
start_time           0
end_time             0
archive_id           0
embed_url            0
sponsors          1162
sponsor_types     1162
race             29872
cycle            29101
subjects         22562
candidates       25127
type                 0
message              0
date_created         0
dtype: int64

In [7]:
ads_df.cycle.unique()

array([2016.,   nan])

In [8]:
ads_df[ads_df.cycle.isnull()].candidates.value_counts()

none                1882
Roy Cooper           960
Pat McCrory          831
Michael Bennet       677
Kasich               333
Gerald Daugherty      62
Name: candidates, dtype: int64

In [9]:
ads_df[ads_df.cycle.isnull()].race.value_counts()

Series([], Name: race, dtype: int64)

Okay to remove null values for "race" and "cycle" since those do not pertain to presidential elections 

In [10]:
# Annie and Phoebe

# Drop any data not related to the 2016 election cycle
print("Shape of data frame before filtering:", ads_df.shape)
ads_df = ads_df[ads_df.race == 'PRES']
print("Shape of data frame after filtering for only PRES race:", ads_df.shape)
ads_df = ads_df[ads_df.cycle == 2016]
ads_df = ads_df[ads_df['end_time'] < '2016-11-09 00:00:00 UTC']
print("Shape of data frame after filtering for only 2016 election cycle prior to the election:", ads_df.shape)

Shape of data frame before filtering: (375350, 19)
Shape of data frame after filtering for only PRES race: (223722, 19)
Shape of data frame after filtering for only 2016 election cycle prior to the election: (223669, 19)


In [11]:
# removed all nulls in crace an dcycle
ads_df.isnull().sum()

id                   0
wp_identifier        0
network              0
location            25
program             14
program_type         0
start_time           0
end_time             0
archive_id           0
embed_url            0
sponsors          1162
sponsor_types     1162
race                 0
cycle                0
subjects         20652
candidates           0
type                 0
message              0
date_created         0
dtype: int64

Variables of interest

Look into and clean the following columns: 
* location
* program_type
* start_time
* end_time
* sponsors
* candidates
* message

In [12]:
# some columns seem to be in various datatypes as most are encapsulated in objects. 
# we may want to check and understand necessary columns
ads_df.dtypes

id                 int64
wp_identifier      int64
network           object
location          object
program           object
program_type      object
start_time        object
end_time          object
archive_id        object
embed_url         object
sponsors          object
sponsor_types     object
race              object
cycle            float64
subjects          object
candidates        object
type              object
message           object
date_created      object
dtype: object

### Create Trump and Clinton identifiers

In [13]:
# Annie's part
# Create columns for candidate contains Hillary Clinton and candidate contains Donald Trump
ads_df["cand_clinton"] = ads_df.candidates.str.contains("clinton", case=False)
ads_df["cand_trump"] = ads_df.candidates.str.contains("trump", case=False)
ads_df.head(5)

,id,wp_identifier,network,location,program,program_type,start_time,end_time,archive_id,embed_url,...,sponsor_types,race,cycle,subjects,candidates,type,message,date_created,cand_clinton,cand_trump
0,1,5643,KLAS,"Las Vegas, NV",8 News Now at 5 PM,news,2016-09-09 00:12:59 UTC,2016-09-09 00:13:29 UTC,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,...,Multiple,PRES,2016.0,"Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,True,False
1,2,5643,WNCN,"Raleigh-Durham-Fayetteville, NC",North Carolina News at 500PM,news,2016-09-06 21:58:25 UTC,2016-09-06 21:58:55 UTC,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,...,Multiple,PRES,2016.0,"Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,True,False
2,3,5643,KLAS,"Las Vegas, NV",Face the Nation,news,2016-09-04 16:24:25 UTC,2016-09-04 16:24:55 UTC,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,...,Multiple,PRES,2016.0,"Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,True,False
3,4,5643,WFLA,"Tampa-St. Petersburg, FL",Days of Our Lives,not news,2016-08-30 17:59:20 UTC,2016-08-30 17:59:50 UTC,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,...,Multiple,PRES,2016.0,"Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,True,False
4,5,5643,KSNV,"Las Vegas, NV",The Tonight Show Starring Jimmy Fallon,news,2016-09-06 07:02:22 UTC,2016-09-06 07:02:52 UTC,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,...,Multiple,PRES,2016.0,"Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,True,False


In [14]:
ads_df['cand_summary'] = np.where(
     ads_df['cand_clinton'], 
    'Clinton', 
     np.where(
        ads_df['cand_trump'], 'Trump', 'Other'
     )
)

In [15]:
pacs = pd.read_csv("data/PACs_categorized.csv")

In [16]:
ads_df = ads_df.merge(pacs, how = 'left', left_on = 'sponsors', right_on = 'unique_values')

In [17]:
ads_df.shape

(223669, 25)

In [18]:
ads_df['supports'] = ads_df['supports'].fillna("Other")

In [19]:
pd.set_option('display.max_columns', None)

ads_df.shape

(223669, 25)

In [20]:
ads_df = ads_df.rename(columns= {'supports':'sponsor_summary'})

In [21]:
ads_df["clinton_sponsored"] = ads_df.sponsor_summary.str.contains("clinton", case=False)
ads_df["trump_sponsored"] = ads_df.sponsor_summary.str.contains("trump", case=False)

In [22]:
ads_df = ads_df.drop(columns = 'counts')

In [23]:
ads_df = ads_df.drop(columns = 'unique_values')

In [24]:
ads_df.head()

,id,wp_identifier,network,location,program,program_type,start_time,end_time,archive_id,embed_url,sponsors,sponsor_types,race,cycle,subjects,candidates,type,message,date_created,cand_clinton,cand_trump,cand_summary,sponsor_summary,clinton_sponsored,trump_sponsored
0,1,5643,KLAS,"Las Vegas, NV",8 News Now at 5 PM,news,2016-09-09 00:12:59 UTC,2016-09-09 00:13:29 UTC,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,2016.0,"Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,True,False,Clinton,Clinton,True,False
1,2,5643,WNCN,"Raleigh-Durham-Fayetteville, NC",North Carolina News at 500PM,news,2016-09-06 21:58:25 UTC,2016-09-06 21:58:55 UTC,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,2016.0,"Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,True,False,Clinton,Clinton,True,False
2,3,5643,KLAS,"Las Vegas, NV",Face the Nation,news,2016-09-04 16:24:25 UTC,2016-09-04 16:24:55 UTC,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,2016.0,"Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,True,False,Clinton,Clinton,True,False
3,4,5643,WFLA,"Tampa-St. Petersburg, FL",Days of Our Lives,not news,2016-08-30 17:59:20 UTC,2016-08-30 17:59:50 UTC,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,2016.0,"Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,True,False,Clinton,Clinton,True,False
4,5,5643,KSNV,"Las Vegas, NV",The Tonight Show Starring Jimmy Fallon,news,2016-09-06 07:02:22 UTC,2016-09-06 07:02:52 UTC,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,2016.0,"Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,True,False,Clinton,Clinton,True,False


### Fix message type for mix

In [25]:
# Phoebe's part
ads_df['message']= ads_df['message'].replace('mix', 'mixed')
ads_df['message'].value_counts(dropna=False)

pro        70070
con        61597
unknown    57317
mixed      34685
Name: message, dtype: int64

### Add Ad Duration column

In [26]:
# change start_time, end_time to "datetime" objects
ads_df['start_time'] = pd.to_datetime(ads_df['start_time'])
ads_df['end_time'] = pd.to_datetime(ads_df['end_time'])

# create a column for ad duration in seconds of each ad shown
ads_df['ad_duration_sec'] = (ads_df.end_time - ads_df.start_time).dt.total_seconds()

In [27]:
# top 5 ad durations
# maybe common for ad runtimes, attention span
# how does ad duration change throughout the presidential election
ads_df.ad_duration_sec.value_counts().head()

30.0    164138
60.0     19113
29.0     10064
32.0      9781
31.0      9166
Name: ad_duration_sec, dtype: int64

### Add State column

In [28]:
# Convert location to strings
ads_df['location'] = ads_df['location'].astype('string')
ads_df.location.value_counts()

Boston, MA/Manchester, NH                            27562
Ceder Rapids-Waterloo-Iowa City-Dublin, Iowa         22384
Las Vegas, NV                                        17850
Des Moines-Ames, Iowa                                17163
Tampa-St. Petersburg, FL                             16688
San Francisco-Oakland-San Jose, CA                   16250
Philadelphia, PA                                     15163
Cleveland, Ohio                                      14039
Sioux City, Iowa                                     12906
Columbia, SC                                         10352
Greenville-Spartanburg, SC/Asheville-Anderson, NC     8558
Raleigh-Durham-Fayetteville,  NC                      8126
Denver, CO                                            6639
Reno, NV                                              6031
Milwaukee, WI                                         4097
Orlando-Daytona Beach-Melbourne, FL                   4027
Phoenix-Prescott, AZ                                  32

In [29]:
# extract states from locations since we care only about that
states = ads_df.location.apply(lambda x : x.split(',')[-1].replace(" ", "") if type(x) == str else 'None')
print(states.unique())
print(len(states.unique()))
# we see Ohio and Iowa not having correct state abbreviation 

['NV' 'NC' 'FL' 'Ohio' 'Iowa' 'PA' 'CO' 'WI' 'CA' 'AZ' 'NH' 'MD' 'NY' 'OH'
 'SC' 'VA' 'None']
17


In [30]:
def correct_state(st): 
    ''' Function to correct state names for Ohio and Iowa'''
    if st == 'Ohio': 
        return 'OH'
    elif st == 'Iowa':
        return 'IA'
    else: 
        return st

In [31]:
# correct poor format states
states = states.apply(lambda x : correct_state(x))
print(states.unique())
print(len(states.unique()))

['NV' 'NC' 'FL' 'OH' 'IA' 'PA' 'CO' 'WI' 'CA' 'AZ' 'NH' 'MD' 'NY' 'SC'
 'VA' 'None']
16


In [32]:
# add states column to our data
ads_df['state'] = states
ads_df.state.value_counts().head()

IA    52453
NH    27562
NV    23881
FL    22073
NC    19434
Name: state, dtype: int64

* We can see Iowa is the state with the most ads shown

### Join Ads Metadata to identify fact checked ads

descriptions of metadata columns can be found here: http://politicaladarchive.org/data/

important columns to add on: 
* reference_count - how many partner orgs fact checked ad
* air_count - how many times ad aired total for States the Internet Archive is tracking (incudes all airings, not just paid airings)
* market_count - how different markets the ad aired in 
* transcript - in case we want to do some NLP on the ad message

In [33]:
# archive_id seems to be the primary key to join on 
cols = ['archive_id', 'reference_count', 'air_count', 'market_count', 'transcript']
merge_df = ads_df.merge(ads_metadata[cols], how='left', on='archive_id')
# create binary flag on whether or not an ad was factchecked
merge_df['fact_checked'] = merge_df.reference_count.apply(lambda x : 1 if x > 0 else 0)

In [34]:
merge_df.head(2)

,id,wp_identifier,network,location,program,program_type,start_time,end_time,archive_id,embed_url,sponsors,sponsor_types,race,cycle,subjects,candidates,type,message,date_created,cand_clinton,cand_trump,cand_summary,sponsor_summary,clinton_sponsored,trump_sponsored,ad_duration_sec,state,reference_count,air_count,market_count,transcript,fact_checked
0,1,5643,KLAS,"Las Vegas, NV",8 News Now at 5 PM,news,2016-09-09 00:12:59+00:00,2016-09-09 00:13:29+00:00,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,2016.0,"Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,True,False,Clinton,Clinton,True,False,30.0,NV,0,1916,6,IThere's a race going on right approve tnow.me...,0
1,2,5643,WNCN,"Raleigh-Durham-Fayetteville, NC",North Carolina News at 500PM,news,2016-09-06 21:58:25+00:00,2016-09-06 21:58:55+00:00,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,2016.0,"Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,True,False,Clinton,Clinton,True,False,30.0,NC,0,1916,6,IThere's a race going on right approve tnow.me...,0


In [35]:
# how many times did a candidates run a fact checked ad
fc = merge_df.groupby(['candidates', 'fact_checked']).agg({'id':'count', 'archive_id':'nunique'}).reset_index(drop=False)
fc[fc.fact_checked == 1].sort_values('id', ascending = False).head()

,candidates,fact_checked,id,archive_id
22,Donald Trump,1,13826,18
54,Hillary Clinton,1,11764,14
27,"Donald Trump, Hillary Clinton",1,9100,5
6,Bernie Sanders,1,6960,7
76,Marco Rubio,1,2282,7


* Donald Trump ran the most fact checked ads
* Candidate value that includes both Trump and Clinton? does it need cleaning? check ads metadata

In [36]:
# what was the messaging type for fact checked ads run? 
fc = merge_df.groupby(['candidates', 'fact_checked', 'message']).agg({'id':'count'}).reset_index(drop=False)
fc[fc.fact_checked == 1].sort_values('id', ascending = False).head()

,candidates,fact_checked,message,id
31,Donald Trump,1,con,10676
41,"Donald Trump, Hillary Clinton",1,mixed,8954
81,Hillary Clinton,1,pro,6551
10,Bernie Sanders,1,unknown,5083
32,Donald Trump,1,pro,3150


* Most of trumps ads that were ran had negative messaging

In [37]:
merge_df.columns

Index(['id', 'wp_identifier', 'network', 'location', 'program', 'program_type',
       'start_time', 'end_time', 'archive_id', 'embed_url', 'sponsors',
       'sponsor_types', 'race', 'cycle', 'subjects', 'candidates', 'type',
       'message', 'date_created', 'cand_clinton', 'cand_trump', 'cand_summary',
       'sponsor_summary', 'clinton_sponsored', 'trump_sponsored',
       'ad_duration_sec', 'state', 'reference_count', 'air_count',
       'market_count', 'transcript', 'fact_checked'],
      dtype='object')

## Add in candidate party affiliations

In [39]:
# data scraped from this website https://www.fec.gov/data/candidates/president/presidential-map/
cand_aff = pd.read_csv('data/2016_candidate_affiliations.csv')
cand_name = cand_aff.cand_nm.apply(lambda x : x.split(" ")[0])
cand_party = cand_aff.cand_nm.apply(lambda x : x.split(" ")[1][1:4])
party = pd.DataFrame([cand_name, cand_party], index=['cand_nm', 'cand_party_long']).T
party['cand_party'] = party.cand_party_long.apply(lambda x : x[0])
party.head()

,cand_nm,cand_party_long,cand_party
0,Clinton,DEM,D
1,Trump,REP,R
2,Sanders,DEM,D
3,Cruz,REP,R
4,Carson,REP,R


In [40]:
party.cand_nm.unique()

array(['Clinton', 'Trump', 'Sanders', 'Cruz', 'Carson', 'Rubio', 'Bush',
       'Kasich', 'Paul', 'Fiorina', 'Johnson', 'Stein', 'Walker',
       'Christie', "O'Malley", 'Graham', 'Huckabee', 'Santorum',
       'McMullin', 'Jindal', 'Perry', 'Lessig', 'Webb', 'Pataki',
       'Gilmore'], dtype=object)

In [41]:
# multi candidates included in ads dataset. Ads related to multiple parties
merge_df.candidates.value_counts()

Donald Trump                                                              56287
Hillary Clinton                                                           49526
Bernie Sanders                                                            31886
Donald Trump, Hillary Clinton                                             22005
Marco Rubio                                                               19894
                                                                          ...  
Donald Trump, Gary Johnson, Gary Johnson, Hillary Clinton, Jill Stein         1
Carly Fiorina, Donald Trump, Hillary Clinton, Ted Cruz                        1
Donald Trump, Hillary Clinton, Jeb Bush, Marco Rubio, Ted Cruz, Carson        1
Hillary Clinton, John Kasich, Ted Cruz                                        1
Ben Carson, Donald Trump, Jeb Bush, Marco Rubio, Ted Cruz                     1
Name: candidates, Length: 74, dtype: int64

In [42]:
# create function to get candidates last names
def cand_last_name(lst):
    if len(lst) == 1: 
        cand = lst[0]
        cand_ln = cand.split(" ")[-1]
        return cand_ln
    
    elif len(lst) > 1:
        names = []
        for cand in lst:
            cand_ln = cand.split(" ")[-1]
            names.append(cand_ln)
        return names

In [43]:
# create function to show how many parties are repesented in the ad
party_list = dict(list(zip(party.cand_nm, party.cand_party)))
def majority_party(name): 
    if type(name) == str:
        # maj_party.append(party_list.get(name))
        return party_list.get(name)
    else:
        # get all parties and join into a sorted list of parties. 
        # can count how many democrats or republicans in 
        cnts = pd.Series(name).apply(lambda x : party_list.get(x)).value_counts().reset_index()
        cnts.columns = ['party', 'cnt']
        party_cnts = cnts.cnt.apply(lambda x : str(x)) + cnts.party
        out = "".join(party_cnts.to_list())
        # maj_party.append(out)
        return out

In [44]:
# candidates refers to the candidates that were mentioned in the ad. 
# we have to look at who funds the ad as who is paying for it
cand_list = merge_df.candidates.apply(lambda x : list(x.split(","))).to_frame()
# # number of candidates per tv ad
cand_list['num_cand'] = cand_list.candidates.apply(lambda x : len(x))
cand_list['cand_ln'] = cand_list.candidates.apply(lambda x : cand_last_name(x))
cand_list['maj_party'] = cand_list.cand_ln.apply(lambda x : majority_party(x))
# # cand_list.head()

In [45]:
merge_df['num_cand'] = cand_list['num_cand']
merge_df['cand_ln'] = cand_list['cand_ln']
merge_df['maj_party'] = cand_list['maj_party']

In [46]:
merge_df.head()

,id,wp_identifier,network,location,program,program_type,start_time,end_time,archive_id,embed_url,sponsors,sponsor_types,race,cycle,subjects,candidates,type,message,date_created,cand_clinton,cand_trump,cand_summary,sponsor_summary,clinton_sponsored,trump_sponsored,ad_duration_sec,state,reference_count,air_count,market_count,transcript,fact_checked,num_cand,cand_ln,maj_party
0,1,5643,KLAS,"Las Vegas, NV",8 News Now at 5 PM,news,2016-09-09 00:12:59+00:00,2016-09-09 00:13:29+00:00,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,2016.0,"Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,True,False,Clinton,Clinton,True,False,30.0,NV,0,1916,6,IThere's a race going on right approve tnow.me...,0,1,Clinton,D
1,2,5643,WNCN,"Raleigh-Durham-Fayetteville, NC",North Carolina News at 500PM,news,2016-09-06 21:58:25+00:00,2016-09-06 21:58:55+00:00,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,2016.0,"Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,True,False,Clinton,Clinton,True,False,30.0,NC,0,1916,6,IThere's a race going on right approve tnow.me...,0,1,Clinton,D
2,3,5643,KLAS,"Las Vegas, NV",Face the Nation,news,2016-09-04 16:24:25+00:00,2016-09-04 16:24:55+00:00,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,2016.0,"Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,True,False,Clinton,Clinton,True,False,30.0,NV,0,1916,6,IThere's a race going on right approve tnow.me...,0,1,Clinton,D
3,4,5643,WFLA,"Tampa-St. Petersburg, FL",Days of Our Lives,not news,2016-08-30 17:59:20+00:00,2016-08-30 17:59:50+00:00,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,2016.0,"Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,True,False,Clinton,Clinton,True,False,30.0,FL,0,1916,6,IThere's a race going on right approve tnow.me...,0,1,Clinton,D
4,5,5643,KSNV,"Las Vegas, NV",The Tonight Show Starring Jimmy Fallon,news,2016-09-06 07:02:22+00:00,2016-09-06 07:02:52+00:00,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,2016.0,"Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,True,False,Clinton,Clinton,True,False,30.0,NV,0,1916,6,IThere's a race going on right approve tnow.me...,0,1,Clinton,D


In [47]:
# how many ads with multiple parties
cand_list.maj_party.value_counts()

R           97879
D           81426
1R1D        22567
2R          10102
1D1R         4397
3R           2664
4R1D         1237
4R            862
1R            836
2R1D          796
2D2R          382
2L1R1D        274
2L             78
2D             23
2R1L           20
2D1R           19
3R1D           15
2L1R            5
1D              5
G               3
5R              1
5R1D            1
2L1R1D1G        1
Name: maj_party, dtype: int64

In [48]:
cand_list.num_cand.value_counts()

1    179384
2     37667
3      3757
4      1599
5      1241
8        20
6         1
Name: num_cand, dtype: int64

In [49]:
# only 20% of the ads we have show more than 2 candidates
cand_list.num_cand.value_counts() / len(cand_list)

1    0.802007
2    0.168405
3    0.016797
4    0.007149
5    0.005548
8    0.000089
6    0.000004
Name: num_cand, dtype: float64

## Create a file with relevant Presedential Election results (2012 & 2016) by state

In [50]:
# From https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/42MVDX

results_df = pd.read_csv('data/1976-2020-president.csv')

In [51]:
results_df.columns

Index(['year', 'state', 'state_po', 'state_fips', 'state_cen', 'state_ic',
       'office', 'candidate', 'party_detailed', 'writein', 'candidatevotes',
       'totalvotes', 'version', 'notes', 'party_simplified'],
      dtype='object')

In [52]:
results_df = results_df[results_df.year < 2020]
results_df = results_df[results_df.year > 2010]

In [53]:
states_in_ads_df = ads_df.state.unique().tolist()

In [54]:
results_df = results_df[results_df.state_po.isin(states_in_ads_df)]

In [55]:
results_df = results_df[results_df.party_detailed.isin(['REPUBLICAN', 'DEMOCRAT'])]

In [56]:
results_df['candidatevotes_pct'] = results_df.candidatevotes / results_df.totalvotes

In [57]:
results_df['state_year'] = results_df.state_po + results_df.year.astype(str)

In [58]:
results_df.head()

,year,state,state_po,state_fips,state_cen,state_ic,office,candidate,party_detailed,writein,candidatevotes,totalvotes,version,notes,party_simplified,candidatevotes_pct,state_year
3088,2012,ARIZONA,AZ,4,86,61,US PRESIDENT,"ROMNEY, MITT",REPUBLICAN,False,1233654,2299254,20210113,NaN,REPUBLICAN,0.536545,AZ2012
3089,2012,ARIZONA,AZ,4,86,61,US PRESIDENT,"OBAMA, BARACK H.",DEMOCRAT,False,1025232,2299254,20210113,NaN,DEMOCRAT,0.445898,AZ2012
3098,2012,CALIFORNIA,CA,6,93,71,US PRESIDENT,"OBAMA, BARACK H.",DEMOCRAT,False,7854285,13038547,20210113,NaN,DEMOCRAT,0.602390,CA2012
3099,2012,CALIFORNIA,CA,6,93,71,US PRESIDENT,"ROMNEY, MITT",REPUBLICAN,False,4839958,13038547,20210113,NaN,REPUBLICAN,0.371204,CA2012
3105,2012,COLORADO,CO,8,84,62,US PRESIDENT,"OBAMA, BARACK H.",DEMOCRAT,False,1323101,2569516,20210113,NaN,DEMOCRAT,0.514922,CO2012


In [59]:
winning_votes = results_df.groupby(['state_po', 'year'], as_index=False).candidatevotes.max()

In [60]:
winning_votes['state_year'] = winning_votes['state_po'] + winning_votes['year'].astype(str)

In [61]:
winning_votes.drop(columns = ['state_po', 'year'], axis = 1, inplace = True)

In [62]:
winning_votes = winning_votes.rename(columns= {'candidatevotes':'winningvotes'})

In [63]:
winning_votes.head()

,winningvotes,state_year
0,1233654,AZ2012
1,1252401,AZ2016
2,7854285,CA2012
3,8753788,CA2016
4,1323101,CO2012


In [64]:
results_merged = results_df.merge(winning_votes, how = 'left', on = 'state_year')

In [65]:
results_merged.head()

,year,state,state_po,state_fips,state_cen,state_ic,office,candidate,party_detailed,writein,candidatevotes,totalvotes,version,notes,party_simplified,candidatevotes_pct,state_year,winningvotes
0,2012,ARIZONA,AZ,4,86,61,US PRESIDENT,"ROMNEY, MITT",REPUBLICAN,False,1233654,2299254,20210113,NaN,REPUBLICAN,0.536545,AZ2012,1233654
1,2012,ARIZONA,AZ,4,86,61,US PRESIDENT,"OBAMA, BARACK H.",DEMOCRAT,False,1025232,2299254,20210113,NaN,DEMOCRAT,0.445898,AZ2012,1233654
2,2012,CALIFORNIA,CA,6,93,71,US PRESIDENT,"OBAMA, BARACK H.",DEMOCRAT,False,7854285,13038547,20210113,NaN,DEMOCRAT,0.602390,CA2012,7854285
3,2012,CALIFORNIA,CA,6,93,71,US PRESIDENT,"ROMNEY, MITT",REPUBLICAN,False,4839958,13038547,20210113,NaN,REPUBLICAN,0.371204,CA2012,7854285
4,2012,COLORADO,CO,8,84,62,US PRESIDENT,"OBAMA, BARACK H.",DEMOCRAT,False,1323101,2569516,20210113,NaN,DEMOCRAT,0.514922,CO2012,1323101


In [66]:
results_merged['winner'] = results_merged['candidatevotes'] == results_merged['winningvotes']

In [67]:
results_merged.head()

,year,state,state_po,state_fips,state_cen,state_ic,office,candidate,party_detailed,writein,candidatevotes,totalvotes,version,notes,party_simplified,candidatevotes_pct,state_year,winningvotes,winner
0,2012,ARIZONA,AZ,4,86,61,US PRESIDENT,"ROMNEY, MITT",REPUBLICAN,False,1233654,2299254,20210113,NaN,REPUBLICAN,0.536545,AZ2012,1233654,True
1,2012,ARIZONA,AZ,4,86,61,US PRESIDENT,"OBAMA, BARACK H.",DEMOCRAT,False,1025232,2299254,20210113,NaN,DEMOCRAT,0.445898,AZ2012,1233654,False
2,2012,CALIFORNIA,CA,6,93,71,US PRESIDENT,"OBAMA, BARACK H.",DEMOCRAT,False,7854285,13038547,20210113,NaN,DEMOCRAT,0.602390,CA2012,7854285,True
3,2012,CALIFORNIA,CA,6,93,71,US PRESIDENT,"ROMNEY, MITT",REPUBLICAN,False,4839958,13038547,20210113,NaN,REPUBLICAN,0.371204,CA2012,7854285,False
4,2012,COLORADO,CO,8,84,62,US PRESIDENT,"OBAMA, BARACK H.",DEMOCRAT,False,1323101,2569516,20210113,NaN,DEMOCRAT,0.514922,CO2012,1323101,True


In [68]:
winners_df = results_merged[results_merged.winner == True].copy()

In [69]:
winners_df.drop(columns = ['state_fips', 'state_ic', 'state_cen', 'writein', 'version', 'notes', 'party_simplified', 'winningvotes', 'winner'], axis = 1, inplace = True)

In [70]:
winners_df = winners_df.sort_values(['state', 'year'], ascending=[True, True])

In [71]:
winners_df.head()

,year,state,state_po,office,candidate,party_detailed,candidatevotes,totalvotes,candidatevotes_pct,state_year
0,2012,ARIZONA,AZ,US PRESIDENT,"ROMNEY, MITT",REPUBLICAN,1233654,2299254,0.536545,AZ2012
30,2016,ARIZONA,AZ,US PRESIDENT,"TRUMP, DONALD J.",REPUBLICAN,1252401,2573165,0.486716,AZ2016
2,2012,CALIFORNIA,CA,US PRESIDENT,"OBAMA, BARACK H.",DEMOCRAT,7854285,13038547,0.602390,CA2012
33,2016,CALIFORNIA,CA,US PRESIDENT,"CLINTON, HILLARY",DEMOCRAT,8753788,14181595,0.617264,CA2016
4,2012,COLORADO,CO,US PRESIDENT,"OBAMA, BARACK H.",DEMOCRAT,1323101,2569516,0.514922,CO2012


In [72]:
df_2012_winners = winners_df[winners_df['year'] == 2012]

In [73]:
df_2012_winners = df_2012_winners.rename(columns= {'candidatevotes':'votes2012', 'totalvotes':'totalvotes2012', 'candidatevotes_pct':'candidatevotes_pct2012', 'candidate':'candidate2012', 'party_detailed':'party2012'})

In [74]:
df_2012_winners.head()

,year,state,state_po,office,candidate2012,party2012,votes2012,totalvotes2012,candidatevotes_pct2012,state_year
0,2012,ARIZONA,AZ,US PRESIDENT,"ROMNEY, MITT",REPUBLICAN,1233654,2299254,0.536545,AZ2012
2,2012,CALIFORNIA,CA,US PRESIDENT,"OBAMA, BARACK H.",DEMOCRAT,7854285,13038547,0.602390,CA2012
4,2012,COLORADO,CO,US PRESIDENT,"OBAMA, BARACK H.",DEMOCRAT,1323101,2569516,0.514922,CO2012
6,2012,FLORIDA,FL,US PRESIDENT,"OBAMA, BARACK H.",DEMOCRAT,4237756,8474179,0.500079,FL2012
8,2012,IOWA,IA,US PRESIDENT,"OBAMA, BARACK H.",DEMOCRAT,822544,1582180,0.519880,IA2012


In [75]:
df_2012_winners.drop(columns = ['year', 'state', 'office', 'state_year'], axis = 1, inplace = True)

In [76]:
winners_results = winners_df.merge(df_2012_winners, how = 'left', on = 'state_po')

In [77]:
winners_results = winners_results[winners_results['year'] == 2016]

In [78]:
winners_results['party_changed'] = winners_results['party_detailed'] != winners_results['party2012']

In [79]:
winners_results['margin_tightened'] = winners_results['candidatevotes_pct'] < winners_results['candidatevotes_pct2012']

In [80]:
winners_results['margin_change'] = winners_results['candidatevotes_pct'] - winners_results['candidatevotes_pct2012']

In [81]:
winners_results['combined'] = winners_results['party_detailed'] + '-' + winners_results['party_changed'].astype(str) + '-' + winners_results['margin_tightened'].astype(str)

In [82]:
def translate_combined(s):
    my_list = []
    my_list = s.split('-')
    if my_list[0] == 'DEMOCRAT':
        if my_list[1] == "True":
            return "FLIPPED DEMOCRAT"
        elif my_list[2] == "True":
            return "DEMOCRAT, MARGIN TIGHTENED"
        else:
            return "DEMOCRAT, MARGIN IMPROVED"
    else:
        if my_list[1] == "True":
            return "FLIPPED REPUBLICAN"
        elif my_list[2] == "True":
            return "REPUBLICAN, MARGIN TIGHTENED"
        else:
            return "REPUBLICAN, MARGIN IMPROVED"

In [83]:
winners_results['result_change_summary'] = winners_results['combined'].map(translate_combined)

In [84]:
winners_results = winners_results.rename(columns= {'state':'state_name'})

In [85]:
winners_results = winners_results.rename(columns= {'state_po':'state'})

In [86]:
winners_results = winners_results.rename(columns= {'candidate':'winner_name'})

In [87]:
winners_results = winners_results.rename(columns= {'candidatevotes_pct':'winner_votes_pct'})

In [88]:
winners_results = winners_results.rename(columns= {'party_detailed':'winner_party'})

In [89]:
winners_results.shape

(15, 20)

In [90]:
winners_results.head(15)

,year,state_name,state,office,winner_name,winner_party,candidatevotes,totalvotes,winner_votes_pct,state_year,candidate2012,party2012,votes2012,totalvotes2012,candidatevotes_pct2012,party_changed,margin_tightened,margin_change,combined,result_change_summary
1,2016,ARIZONA,AZ,US PRESIDENT,"TRUMP, DONALD J.",REPUBLICAN,1252401,2573165,0.486716,AZ2016,"ROMNEY, MITT",REPUBLICAN,1233654,2299254,0.536545,False,True,-0.049829,REPUBLICAN-False-True,"REPUBLICAN, MARGIN TIGHTENED"
3,2016,CALIFORNIA,CA,US PRESIDENT,"CLINTON, HILLARY",DEMOCRAT,8753788,14181595,0.617264,CA2016,"OBAMA, BARACK H.",DEMOCRAT,7854285,13038547,0.602390,False,False,0.014874,DEMOCRAT-False-False,"DEMOCRAT, MARGIN IMPROVED"
5,2016,COLORADO,CO,US PRESIDENT,"CLINTON, HILLARY",DEMOCRAT,1338870,2780220,0.481570,CO2016,"OBAMA, BARACK H.",DEMOCRAT,1323101,2569516,0.514922,False,True,-0.033352,DEMOCRAT-False-True,"DEMOCRAT, MARGIN TIGHTENED"
7,2016,FLORIDA,FL,US PRESIDENT,"TRUMP, DONALD J.",REPUBLICAN,4617886,9420039,0.490219,FL2016,"OBAMA, BARACK H.",DEMOCRAT,4237756,8474179,0.500079,True,True,-0.009859,REPUBLICAN-True-True,FLIPPED REPUBLICAN
9,2016,IOWA,IA,US PRESIDENT,"TRUMP, DONALD J.",REPUBLICAN,800983,1565580,0.511621,IA2016,"OBAMA, BARACK H.",DEMOCRAT,822544,1582180,0.519880,True,True,-0.008260,REPUBLICAN-True-True,FLIPPED REPUBLICAN
11,2016,MARYLAND,MD,US PRESIDENT,"CLINTON, HILLARY",DEMOCRAT,1677928,2781446,0.603257,MD2016,"OBAMA, BARACK H.",DEMOCRAT,1677844,2707327,0.619742,False,True,-0.016484,DEMOCRAT-False-True,"DEMOCRAT, MARGIN TIGHTENED"
13,2016,NEVADA,NV,US PRESIDENT,"CLINTON, HILLARY",DEMOCRAT,539260,1125385,0.479178,NV2016,"OBAMA, BARACK H.",DEMOCRAT,531373,1014918,0.523562,False,True,-0.044384,DEMOCRAT-False-True,"DEMOCRAT, MARGIN TIGHTENED"
15,2016,NEW HAMPSHIRE,NH,US PRESIDENT,"CLINTON, HILLARY",DEMOCRAT,348526,744296,0.468263,NH2016,"OBAMA, BARACK H.",DEMOCRAT,369561,710972,0.519797,False,True,-0.051534,DEMOCRAT-False-True,"DEMOCRAT, MARGIN TIGHTENED"
17,2016,NEW YORK,NY,US PRESIDENT,"CLINTON, HILLARY",DEMOCRAT,4379789,7802084,0.561361,NY2016,"OBAMA, BARACK H.",DEMOCRAT,4324228,7116784,0.607610,False,True,-0.046248,DEMOCRAT-False-True,"DEMOCRAT, MARGIN TIGHTENED"
19,2016,NORTH CAROLINA,NC,US PRESIDENT,"TRUMP, DONALD J.",REPUBLICAN,2362631,4741564,0.498281,NC2016,"ROMNEY, MITT",REPUBLICAN,2270395,4505372,0.503931,False,True,-0.005650,REPUBLICAN-False-True,"REPUBLICAN, MARGIN TIGHTENED"


In [91]:
winners_results.to_csv("data/election_results_by_state_2012_2016.csv", index=False)

In [92]:
winners_results.drop(columns = ['year', 'state_name', 'office', 'candidatevotes', 'totalvotes', 'state_year', 'candidate2012', 'party2012', 'votes2012', 'totalvotes2012', 'candidatevotes_pct2012', 'party_changed', 'margin_tightened', 'combined'], axis = 1, inplace = True)

In [93]:
winners_results.drop(columns = ['margin_change'], axis=1, inplace=True)

In [94]:
winners_results.head()

,state,winner_name,winner_party,winner_votes_pct,result_change_summary
1,AZ,"TRUMP, DONALD J.",REPUBLICAN,0.486716,"REPUBLICAN, MARGIN TIGHTENED"
3,CA,"CLINTON, HILLARY",DEMOCRAT,0.617264,"DEMOCRAT, MARGIN IMPROVED"
5,CO,"CLINTON, HILLARY",DEMOCRAT,0.481570,"DEMOCRAT, MARGIN TIGHTENED"
7,FL,"TRUMP, DONALD J.",REPUBLICAN,0.490219,FLIPPED REPUBLICAN
9,IA,"TRUMP, DONALD J.",REPUBLICAN,0.511621,FLIPPED REPUBLICAN


In [95]:
merge_df.head()

,id,wp_identifier,network,location,program,program_type,start_time,end_time,archive_id,embed_url,sponsors,sponsor_types,race,cycle,subjects,candidates,type,message,date_created,cand_clinton,cand_trump,cand_summary,sponsor_summary,clinton_sponsored,trump_sponsored,ad_duration_sec,state,reference_count,air_count,market_count,transcript,fact_checked,num_cand,cand_ln,maj_party
0,1,5643,KLAS,"Las Vegas, NV",8 News Now at 5 PM,news,2016-09-09 00:12:59+00:00,2016-09-09 00:13:29+00:00,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,2016.0,"Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,True,False,Clinton,Clinton,True,False,30.0,NV,0,1916,6,IThere's a race going on right approve tnow.me...,0,1,Clinton,D
1,2,5643,WNCN,"Raleigh-Durham-Fayetteville, NC",North Carolina News at 500PM,news,2016-09-06 21:58:25+00:00,2016-09-06 21:58:55+00:00,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,2016.0,"Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,True,False,Clinton,Clinton,True,False,30.0,NC,0,1916,6,IThere's a race going on right approve tnow.me...,0,1,Clinton,D
2,3,5643,KLAS,"Las Vegas, NV",Face the Nation,news,2016-09-04 16:24:25+00:00,2016-09-04 16:24:55+00:00,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,2016.0,"Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,True,False,Clinton,Clinton,True,False,30.0,NV,0,1916,6,IThere's a race going on right approve tnow.me...,0,1,Clinton,D
3,4,5643,WFLA,"Tampa-St. Petersburg, FL",Days of Our Lives,not news,2016-08-30 17:59:20+00:00,2016-08-30 17:59:50+00:00,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,2016.0,"Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,True,False,Clinton,Clinton,True,False,30.0,FL,0,1916,6,IThere's a race going on right approve tnow.me...,0,1,Clinton,D
4,5,5643,KSNV,"Las Vegas, NV",The Tonight Show Starring Jimmy Fallon,news,2016-09-06 07:02:22+00:00,2016-09-06 07:02:52+00:00,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,2016.0,"Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,True,False,Clinton,Clinton,True,False,30.0,NV,0,1916,6,IThere's a race going on right approve tnow.me...,0,1,Clinton,D


In [96]:
merge_df = merge_df.merge(winners_results, how = 'left', on = 'state')

In [97]:
merge_df.head()

,id,wp_identifier,network,location,program,program_type,start_time,end_time,archive_id,embed_url,sponsors,sponsor_types,race,cycle,subjects,candidates,type,message,date_created,cand_clinton,cand_trump,cand_summary,sponsor_summary,clinton_sponsored,trump_sponsored,ad_duration_sec,state,reference_count,air_count,market_count,transcript,fact_checked,num_cand,cand_ln,maj_party,winner_name,winner_party,winner_votes_pct,result_change_summary
0,1,5643,KLAS,"Las Vegas, NV",8 News Now at 5 PM,news,2016-09-09 00:12:59+00:00,2016-09-09 00:13:29+00:00,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,2016.0,"Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,True,False,Clinton,Clinton,True,False,30.0,NV,0,1916,6,IThere's a race going on right approve tnow.me...,0,1,Clinton,D,"CLINTON, HILLARY",DEMOCRAT,0.479178,"DEMOCRAT, MARGIN TIGHTENED"
1,2,5643,WNCN,"Raleigh-Durham-Fayetteville, NC",North Carolina News at 500PM,news,2016-09-06 21:58:25+00:00,2016-09-06 21:58:55+00:00,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,2016.0,"Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,True,False,Clinton,Clinton,True,False,30.0,NC,0,1916,6,IThere's a race going on right approve tnow.me...,0,1,Clinton,D,"TRUMP, DONALD J.",REPUBLICAN,0.498281,"REPUBLICAN, MARGIN TIGHTENED"
2,3,5643,KLAS,"Las Vegas, NV",Face the Nation,news,2016-09-04 16:24:25+00:00,2016-09-04 16:24:55+00:00,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,2016.0,"Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,True,False,Clinton,Clinton,True,False,30.0,NV,0,1916,6,IThere's a race going on right approve tnow.me...,0,1,Clinton,D,"CLINTON, HILLARY",DEMOCRAT,0.479178,"DEMOCRAT, MARGIN TIGHTENED"
3,4,5643,WFLA,"Tampa-St. Petersburg, FL",Days of Our Lives,not news,2016-08-30 17:59:20+00:00,2016-08-30 17:59:50+00:00,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,2016.0,"Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,True,False,Clinton,Clinton,True,False,30.0,FL,0,1916,6,IThere's a race going on right approve tnow.me...,0,1,Clinton,D,"TRUMP, DONALD J.",REPUBLICAN,0.490219,FLIPPED REPUBLICAN
4,5,5643,KSNV,"Las Vegas, NV",The Tonight Show Starring Jimmy Fallon,news,2016-09-06 07:02:22+00:00,2016-09-06 07:02:52+00:00,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,2016.0,"Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,True,False,Clinton,Clinton,True,False,30.0,NV,0,1916,6,IThere's a race going on right approve tnow.me...,0,1,Clinton,D,"CLINTON, HILLARY",DEMOCRAT,0.479178,"DEMOCRAT, MARGIN TIGHTENED"


In [98]:
# Output cleaned .csv file
# Note that we can keep adding steps in this workbook to improve on the cleaned file

merge_df.to_csv("data/political_tv_ad_archive_CLEANED.csv", index=False)